<a href="https://colab.research.google.com/github/monacofj/moeabench/blob/add-test/misc/benchmark_gallery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MoeaBench: Scientific Audit Confrontation
This notebook provides a systematic visual audit comparing the **Legacy Heuristic Fronts (v0.6.x)** against the **Analytical Ground Truth (v0.7.5)**.

It demonstrates the jump in mathematical precision and identifies the geometric displacement of legacy truths.

In [ ]:
# Setup Colab: Clone, Enter and Install
import os
if 'google.colab' in str(get_ipython()):
    if not os.path.exists('moeabench'):
        !git clone https://github.com/monacofj/moeabench.git
    %cd moeabench
    !pip install -e .
    print("Environment ready.")

In [ ]:
import os
import sys
import pandas as pd
import numpy as np

if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

import MoeaBench as mb
from MoeaBench.core import SmartArray

## Visual Auditor Helper
Logic to resolve mismatched legacy paths, handle missing analytical sampling (DTLZ8), and plot the confrontation.

In [ ]:
def audit_confrontation(mop_name, M=3):
    # 1. Setup Problem Instance
    exp = mb.experiment()
    if mop_name.startswith("DPF"):
        exp.mop = getattr(mb.mops, mop_name)(M=M, D=M-1)
    else:
        exp.mop = getattr(mb.mops, mop_name)(M=M)
    
    # 2. Setup New Truth (v0.7.5)
    # Try analytical generation first; fallback to CSV for complex cases (DTLZ8)
    try:
        f_gt_pop = exp.optimal(n_points=500)
        F_gt = f_gt_pop.objs
    except (NotImplementedError, AttributeError):
        # Load from frozen Ground Truth directory
        gt_file = os.path.join("tests/ground_truth", f"{mop_name}_{M}_optimal.csv")
        if os.path.exists(gt_file):
            F_gt = pd.read_csv(gt_file, header=None).values
        else:
            print(f"Critical: Analytical truth not available for {mop_name}.")
            return
            
    gt_smart = SmartArray(F_gt, name="v0.7.5 Ground Truth", 
                          label="Referência Científica")
    
    # 3. Resolve and Load Legacy (v0.6.x)
    prefix = "lg__" if "DTLZ" in mop_name else "lg_"
    leg_filename = f"{prefix}{mop_name}_{M}_opt_front.csv"
    leg_path = os.path.join("tests/audit_data", f"legacy_{mop_name}", leg_filename)
    
    if os.path.exists(leg_path):
        F_leg = pd.read_csv(leg_path, header=None).values
        leg_smart = SmartArray(F_leg, name="v0.6.x Legacy", 
                               label="Heurístico (Rastro)")
        
        # 4. View Confrontation
        mb.view.topo_shape(gt_smart, leg_smart, 
                           title=f"{mop_name} (M={M}): Confronto de Verdades")
    else:
        print(f"Warning: Legacy data for {mop_name} not found. Showing only GT.")
        mb.view.topo_shape(gt_smart, title=f"{mop_name} (M={M}): Ground Truth")

## 1. DTLZ Audit Loop

In [ ]:
for i in range(1, 10):
    print(f"Rendering {f'DTLZ{i}'}...")
    audit_confrontation(f"DTLZ{i}")

## 2. DPF Audit Loop

In [ ]:
for i in range(1, 6):
    print(f"Rendering {f'DPF{i}'}...")
    audit_confrontation(f"DPF{i}")